In [1]:
import glob
import re
import json

In [2]:
files = glob.glob("tesseract-ocr/*.txt")
len(files)

1316

In [3]:
data = []
for file in files:
    with open(file, "r", encoding="utf-8") as f:
        content = f.read()
    data.append(content)

In [4]:
len(data)

1316

In [11]:
for file in files:
    collection = {}
    with open(file, "r", encoding="utf-8") as f:
        item = f.read()
    file = file.split("\\")[-1].replace(".txt", "")
    if file.split(".")[1] == "562":
        print("found")
        break
    questions = []
    answers = []
    headers = []
    misses = []
    item = re.sub("\nA:", "\n\nA:", item)
    item = re.sub("\nAnswer:", "\n\nAnswer:", item)
    item = re.sub("\nQ:", "\n\nQ:", item)
    item = re.sub("\nQuestion:", "\n\nQuestion:", item)
    item = re.sub("\n\n\n", "\n\n", item)
    item = re.sub("http://collections.ushmm.org", "", item)
    item = re.sub("\nContact reference@ushmm.org for further information about tis collection\n", "", item)
    item = re.sub("Contact reference@ushmm.org for further information about this collection", "", item)
    item = re.sub("This is a verbatim transcript of spoken word. It is not the primary source, and it has not been checked for spelling or accuracy.", "", item)
    item = re.sub("\nInterview with.*\n", "", item)
    item = re.sub("\nUSHMM Archives.*\n", "", item)
    item = re.sub("\n\d{1,2}:\d{1,2}:\d{1,2}", "", item)
    
    while "\n\n\n" in item:
        item = item.replace("\n\n\n", "\n\n")
    sections = item.split("\n\n")
    sequence = []
    collection = {"rg": file, "sequence": [], "questions":[], "answers": [], "headers": [], "paragraph_breaks": []}
    # print(len(lines))
    for section in sections:
        lines = []
        for line in section.split("\n"):
            if "USHMM" not in line:
                lines.append(line)
            else:
                headers.append(line)
        section = "\n".join(lines)       
        if "Q:" in section or "question:" in section.lower():
            collection["questions"].append(section)
            sequence.append(section)
        elif "A:" in section or "answer:" in section.lower():
            collection["answers"].append(section)
            sequence.append(section)
        else:
            if collection["answers"]:
                sequence[-1] = f'{sequence[-1]}\n{section}'
                collection["answers"][-1] = f'{collection["answers"][-1]}\n{section}'
            else:
                collection["paragraph_breaks"].append(section)
    collection["sequence"] = sequence
    with open(f"clean_ocr/{file}.json", "w", encoding="utf-8") as f:
        json.dump(collection, f, indent=4)
            

found


In [90]:
collection

{}